In [ ]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import os
import matplotlib.pyplot as plt

In [ ]:
# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load class names from classes.txt
def load_class_names(classes_file):
    try:
        with open(classes_file, 'r') as f:
            classes = [line.strip().split(' ', 1)[1] for line in f.readlines()]
        return classes
    except Exception as e:
        print(f"Error loading classes.txt: {str(e)}")
        return None

# Load and preprocess the image
def load_image(image_path):
    try:
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        image = Image.open(image_path).convert('RGB')
        image = transform(image).unsqueeze(0)
        return image.to(device)
    except Exception as e:
        print(f"Error loading image: {str(e)}")
        return None

# Load pre-trained ResNet-50 model
def load_model(model_path, num_classes=102):
    try:
        model = models.resnet50(weights=None)
        model.fc = nn.Linear(model.fc.in_features, num_classes)
        model.load_state_dict(torch.load(model_path, map_location=device))
        model = model.to(device)
        model.eval()
        return model
    except Exception as e:
        print(f"Error loading model: {str(e)}")
        return None

In [ ]:
# Predict pest class
def predict_pest(image_path, model, class_names):
    if model is None or class_names is None:
        print("Cannot predict: Model or class names not loaded.")
        return None
    image = load_image(image_path)
    if image is None:
        return None
    try:
        with torch.no_grad():
            outputs = model(image)
            _, predicted = torch.max(outputs, 1)
            predicted_class = class_names[predicted.item()]
        return predicted_class
    except Exception as e:
        print(f"Error during prediction: {str(e)}")
        return None

In [ ]:

def main():
    # Paths (update these based on your setup)
    classes_file = "/root/.cache/kagglehub/datasets/rtlmhjbn/ip02-dataset/versions/1/classes.txt"  # Update with path to classes.txt
    model_path = "/content/resnet50_ip102.pth"  # Path to saved model
    image_path = '/root/.cache/kagglehub/datasets/rtlmhjbn/ip02-dataset/versions/1/classification/train/0/00045.jpg'
    # Validate inputs
    if not os.path.exists(image_path):
        print("Error: Image file does not exist.")
        return
    if not os.path.exists(classes_file):
        print("Error: classes.txt file not found.")
        return
    if not os.path.exists(model_path):
        print("Error: Model file not found.")
        return

    # Load class names and model
    class_names = load_class_names(classes_file)
    if class_names is None:
        return
    model = load_model(model_path, num_classes=len(class_names))

    # Predict
    predicted_class = predict_pest(image_path, model, class_names)
    if predicted_class:
       print(f"Predicted pest class: {predicted_class}")
       img = Image.open(image_path)
       plt.imshow(img)
       plt.title(f"Predicted pest class: {predicted_class}")
       plt.axis('off') # Hide axes
       plt.show()

In [ ]:
main()